In [4]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
print (wn.synsets('cat', 'n'))

print (wn.synsets('dog', 'n'))
 
print (wn.synsets('feline', 'n'))
 
print (wn.synsets('mammal', 'n'))

[Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01')]
[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01')]
[Synset('feline.n.01')]
[Synset('mammal.n.01')]


In [6]:
cat = wn.synsets('cat', 'n')[0]     
print (cat.lemmas()[0].count() )

18


In [7]:
dog = wn.synsets('dog', 'n')[0]          
feline = wn.synsets('feline', 'n')[0]    
mammal = wn.synsets('mammal', 'n')[0]    

In [8]:
for synset in [dog, feline, mammal]:
    print ("Similarity(%s, %s) = %s" % (cat, synset, cat.wup_similarity(synset)))

Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.8571428571428571
Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.9629629629629629
Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.8333333333333334


In [9]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for syn1 in synsets1:
      arr_simi_score = []
      # print('=========================================')
      # print(syn1)
      # print('----------------')
      for syn2 in synsets2:
        # print(syn2)
        simi_score = syn1.path_similarity(syn2)
        # print(simi_score)
        if simi_score is not None:
          arr_simi_score.append(simi_score)
      # print('----------------')
      # print(arr_simi_score)
      if(len(arr_simi_score) > 0):
        best = max(arr_simi_score)
        # print(best)
        score += best
        count += 1

    # Average the values
    # print('score: ', score)
    # print('count: ', count)
    score /= count
    return score

sentences = [
    "Dogs are awesome.",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "Cats are beautiful animals.",
]
 
focus_sentence = "Cats are beautiful animals."
 
for sentence in sentences:
    print ("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence)))
    print ("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence)))

 
# Similarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.511111111111
# Similarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.666666666667
 
# Similarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.833333333333
# Similarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.833333333333
 
# Similarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.483333333333
# Similarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.4
 
# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Similarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.5111111111111111
Similarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.6666666666666666
Similarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.8333333333333334
Similarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.8333333333333334
Similarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.48333333333333334
Similarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.4
Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0


In [10]:
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 
 
for sentence in sentences:
    print ("SymmetricSimilarity(\"%s\", \"%s\") = %s" % (
        focus_sentence, sentence, symmetric_sentence_similarity(focus_sentence, sentence)))
    print ("SymmetricSimilarity(\"%s\", \"%s\") = %s" % (
        sentence, focus_sentence, symmetric_sentence_similarity(sentence, focus_sentence)))

SymmetricSimilarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.5888888888888888
SymmetricSimilarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.5888888888888888
SymmetricSimilarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.8333333333333334
SymmetricSimilarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.8333333333333334
SymmetricSimilarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.44166666666666665
SymmetricSimilarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.44166666666666665
SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
